<h2>Load and Prepare Data</h2>

We're also going to visualize the data just to see what's going on and how everything correlates with our y values

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv("./coffee_shop_revenue.csv")

y = data["Daily_Revenue"]

X = data.drop("Daily_Revenue", axis=1)

# Combine features and target into one DataFrame for easy filtering
data = pd.concat([X, y], axis=1)

# Drop rows where the target variable is NaN
cleaned_data = data.dropna()

# Split the data back into features (X) and target (y)
X = cleaned_data.iloc[:, :-1]
y = cleaned_data.iloc[:, -1]

# Display the number of rows removed
rows_removed = len(data) - len(cleaned_data)

Rows removed: 0


In [2]:
# sns.pairplot(data, kind='scatter', plot_kws={'alpha': 0.4})

# sns.lmplot(x="Average_Order_Value", y="Daily_Revenue", data = data, scatter_kws={'alpha': 0.1})

<h2>Splitting data into training and test sets</h2>

In [15]:
from sklearn.model_selection import train_test_split

# Do a 70/30 split (e.g., 70% train, 30% other)
X_train, X_leftover, y_train, y_leftover = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,    # for reproducibility
    shuffle=True,       # whether to shuffle the data before splitting
)

# Split the remaining 30% into validation/testing (15%/15%)
X_val, X_test, y_val, y_test = train_test_split(
    X_leftover, y_leftover,
    test_size=0.5,
    random_state=42,
    shuffle=True,
)

# Compute statistics for X (features)
X_mean = X_train.mean(axis=0)  # Mean of each feature
X_std = X_train.std(axis=0)    # Standard deviation of each feature

# Standardize X
X_train = (X_train - X_mean) / X_std
X_val = (X_val - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

# Compute statistics for y (targets)
y_mean = y_train.mean()  # Mean of target
y_std = y_train.std()    # Standard deviation of target

# Standardize y
y_train = (y_train - y_mean) / y_std
y_val = (y_val - y_mean) / y_std
y_test = (y_test - y_mean) / y_std

X_mean

Number_of_Customers_Per_Day    273.730714
Average_Order_Value              6.293300
Operating_Hours_Per_Day         11.666429
Number_of_Employees              7.948571
Marketing_Spend_Per_Day        253.041707
Location_Foot_Traffic          532.693571
dtype: float64

<h2>Train model</h2>

In [4]:
# from main import *
# model = LinearRegressionModel(6, 1, 4)

# epoch_count, loss_values, test_loss_values = model.fit(model, 
#                                                        torch.tensor(X_train.values, dtype=torch.float), 
#                                                        torch.tensor(y_train.values, dtype=torch.float), 
#                                                        125, 
#                                                        torch.tensor(X_val.values, dtype=torch.float), 
#                                                        torch.tensor(y_val.values, dtype=torch.float))

In [5]:
# plot_loss(epoch_count, loss_values, test_loss_values, "Validation Loss")

<h2>Save model</h2>

In [6]:
# # Saving our PyTorch model
# from pathlib import Path

# # 1. Create model's directory
# MODEL_PATH = Path("models")
# MODEL_PATH.mkdir(parents=True, exist_ok=True)

# # 2. Create model save path
# MODEL_NAME = "coffee_prediction_model.pth"
# MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# # 3. Save the model state dict
# torch.save(obj=model.state_dict(),f=MODEL_SAVE_PATH)

In [7]:
from main import *
tail = X.head(1)
tail = (tail - X_mean) / X_std
dollars = generate(torch.tensor(tail.values, dtype=torch.float))
dollars.data[0]

tensor([-0.9514])

<h2>Evaluate the Model</h2>

In [13]:
loss_val = evaluate(torch.tensor(X_test.values, dtype=torch.float), torch.tensor(y_test.values, dtype=torch.float))
print(f"Loss value for testing data: {loss_val}")

Loss value for testing data: 0.05561050400137901


<h2>Code to generate the dollar amount</h2>

In [14]:
from main import *
import pandas as pd
# customers, avg order, op hours, emploi, market spend, foot traff
head = [[210, 9.86, 7, 10, 456, 673]]
head = pd.DataFrame(head, columns=['Number_of_Customers_Per_Day', 'Average_Order_Value', 'Operating_Hours_Per_Day', 'Number_of_Employees', 'Marketing_Spend_Per_Day', 'Location_Foot_Traffic'])
head = (head - X_mean) / X_std
money = generate(torch.tensor(head.values, dtype=torch.float))
print(money.data[0].item() * y_std + y_mean)

# 210	9.86	7	10	456	673	2397.9

2810.4792357162114


In [ ]:
# I need to print out a column of the test printed out in a table compared to the true y values